__Function__: IMU_Roll_Pitch( *...imu.nc data file* , *associated video file\** )


__Inputs__: xarray dataset containing acc and gyro data, associated world cam video* (this feature is still being worked on, so for now only input the former)

__Outputs__: *ndarray* of [Roll, Pitch, Time]

*before importing:*
1. download parallax repository: https://github.com/wehr-lab/autopilot/tree/parallax
2. `pip install -e ./autopilot/` (if you navigate to autopilot folder then use 'pip install -e .' you should get the same result--unfortunately I forgot to write this part down when we did it, so this line may be a bit off)
3. configure autopilot: `python3 -m autopilot.setup.setup_autopilot`
- Make terminal big enough to run GUI...select terminal...no need to edit anything just keep pressing 'ok'...choose the autopilot install option...

*You are set to import! and you wont ever have to do the above again :D*


In [3]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import typing

from time import time
from scipy.spatial import distance
from scipy.spatial.transform import Rotation as R
from scipy.optimize import curve_fit

from autopilot.transform.geometry import IMU_Orientation
IMU = IMU_Orientation()

In [4]:
def IMU_Roll_Pitch(fname):
# 1. opening the data
    with xr.open_dataset(fname) as ds: 
        #print(ds.keys())
        data = ds.IMU_data ### saves the Datasets DataArray

# 2. Time
    timestamps = data.coords['timestamps'] ### seconds past midnight

    diff = np.diff(timestamps) ### gives the difference between timestamps (in seconds)

    t = np.zeros([len(timestamps),1]) ### gives a list of seconds over trial period
    for i in range(len(diff)):
        t[1+i] = t[i] + diff[i]

# 3. Isolating acceleration and gyro measurments (volts)
    acc = data[:,0:3]
    gyro = data[:,3:6]

# 4. Converting acceleration and gyro units
    gyro = (gyro-np.mean(gyro))*(400) ### 1V = 400deg/sec ... mean(gyro) centers data around 0

    acc = (acc-2.5)*1.6 # convert to: g = m/s^2

# 5. Saving Acc and Gyro as ndarrays
    acc = acc.data
    gyro = gyro.data
    
# 6. Collect Roll & Pitch
    if len(acc) == len(gyro):
        roll_pitch = np.zeros([len(acc),2])
        for x in range(len(acc)):
            roll_pitch[x,:] = IMU.process((acc[x,:],gyro[x,:]))### update by row
        #return(roll_pitch)
    else:
        print('acc or gyro error: input arguments must be of equal size')
        
# 7. Spit out Roll_Pitch_Time
    RPT = np.append(roll_pitch, t , axis=1)
    return(RPT)

In [5]:
IMU_Roll_Pitch('')

array([[ 1.80205229e+01, -3.56293076e-01,  0.00000000e+00],
       [ 2.30940049e+01, -1.74619639e+00,  3.33333333e-03],
       [ 2.50949977e+01, -2.04144881e+00,  6.66666667e-03],
       ...,
       [ 3.38395490e+01,  9.99592712e+00,  1.29036667e+03],
       [ 3.39569922e+01,  9.77893972e+00,  1.29037000e+03],
       [ 3.40139302e+01,  9.73006593e+00,  1.29037333e+03]])

# About IMU_Orientation:
__Function__: IMU_Orientation


__Inputs__: tuple of ( accelerometer [x,y,z] , gyroscope [x,y,z] ) *ndarrays* 
- can also pass a single *ndarray* with just accelerometer [x,y,z]

__Outputs__: *ndarray* of filtered [Roll, Pitch] in degrees

## Things to look into:
- Jonny's Autopilot configuration (necessary for IMU_Orientation import) says that it is only supported on linux and macOS
- There is still some cleaning/optimizing to be done with the above...but for purly roll/pitch data collection it should be set.
- When you run a single dataset through the function mutliple times you will most likely see variable outputs...I think this has something to do with the Kalman filter? Not sure how we should go about this/how it might affect future calculations from this data.